# Knuth Morris Pratt algorithm

## Introduction

In string computation, the **exact pattern matching problem** is the problem of finding all the occurences of a pattern (string) P, in a text (string) S, where usually P is much shorter than S.

For example the pattern could be the world “stella” and the text the whole Divina Commedia, or P can be the CCATTGTG motif and the text the human genome.

One strategy to speed up the computation is to create an **index** on the pattern P and use this index to scan the text S in a more efficient way.

The **Knuth-Morris-Pratt algorithm** uses this approach. It first of all builds an index on P and then uses it to scan S, applying simple rules to the index to decide how to shift the pattern.

### A naive approach

The **naive algorithm** for pattern matching is to look for a word match at each index of S. So, at each position of S the algorithm check if there is a match with the first character of P. If there is, then the algorithm tests the other characters in P by comparing them to successive characters of S. If all successive characters match there is an occurence found.

The worst case in the use of the naive approach is if the two strings match in all but the last letter, for example S is a string of length n full of "A" and P is a string of length k full of "A", but the last character is different. In this case the algorithm compares k characters at each position in string S, so the time complexity is *O(kn)*

## KMP algorithm
The KMP algorithm, before scan S, creates an **index of P** which allows to use of **previous match information** in order to skip matches. In the example discussed above, where S is "AAAAAA...A" and P is "AAA...AB", when KMP sees a trial match fail on the last character of P, in the next iteration it will know that the first k-2 characters at the new position already match so it skip all of these comparisons.

### Index building
The purpose of index building is to **find prefixes** of P that are also present in other positions of P.

In order to do this the algorithm scans P and for each position pos check if the suffix \[pos:k\] contains a prefix which is also a prefix for P. This **information** is then used in the KMP algorithm because it say the next position in S where we can start to match P and the number of matches that the algorithm can skip, because they are already checked in the previous iteration.

In [1]:
import numpy as np

#This function build the index for the pattern string, required in order to run the Knuth Morris Pratt algorithm
def buildIndex(pattern):
    #Create index array with one element more than pattern string
    index = np.empty(len(pattern) + 1, dtype = np.int32)
    len(index)
    
    #Initialize first element to -1
    index[0] = -1
    
    cnd = 0 #Zero base index in pattern of the next character of the current longest prefix
    
    for pos in range(1,len(pattern)):
        if pattern[pos] == pattern[cnd]:
            index[pos] = index[cnd]
        else:
            index[pos] = cnd
            while cnd >= 0 and pattern[pos] != pattern[cnd]:
                cnd = index[cnd]
        cnd = cnd + 1
        
    index[len(pattern)] = cnd
    
    return index

In [2]:
pattern = "ABADE"

print('The index of pattern "{}" is {}'.format(pattern, buildIndex("ABADE")))

The index of pattern "ABADE" is [-1  0 -1  1  0  0]


#### Efficency of index building
Time and space **complexity** of table algorithm is *O(k)*, where k is length of P. The index is constructed so that if a match which had begun at S\[m\] fails while comparing S\[m + i\] to P\[i\], then the next possible match must begin at S\[m + (i - index\[i\])\]. In particular, the next possible match must occur at a higher index than m, so that index\[i\] < i.

* **Outer loop**: k-1 iterations.
* **Inner loop**: cnd is initialized to 0 and gets increased by at most 1 in each outer loop iteration; index\[cnd\] is always less than cnd, so cnd gets decreased by at least 1 in each inner loop iteration; the inner loop condition is cnd ≥ 0. So the inner loop can execute at most as many times in total, as the outer loop has executed – each decrease of cnd by 1 in the inner loop needs to have a corresponding increase by 1 in the outer loop. Since the outer loop takes k-1 iterations, the inner loop can take no more than k-1 iterations in total.

Overall, there are at most 2k-2 iterations, so the time complexity is *O(k)*

### KMP search algorithm
KMP algorithm first of all builds the **index of P** and then uses it to search occurences of P in S.

To illustrate the process of the algorithm, we notice that the algorithm is in a state determined by two integers:
* m, denoting the position within S where the prospective match for P begins,
* i, denoting the index of the currently considered character in P.

In each step the algorithm compares S\[m+i\] with W\[i\] and increments i if they are equal. When there is not a match, then the **next possible occurrence** will start at index m+i-index\[i\] in S, but the first index\[i\] characters are already matched in the previous iteration, so we continue searching from P\[index\[i\]\] and S\[i\].

In this way, the algorithm **bypasses re-examination** of previously matched characters

In [3]:
#This function run the Knuth Morris Pratt algorithm, which finds all the occurrences of pattern in text.
#In case of empty pattern, return empty list
def kmpSearch(text, pattern):
    #If empty pattern, return empty list
    if(not pattern):
        return []
    
    j = 0 #Position of the current character in text
    k = 0 #Position of the current character in pattern
    index = buildIndex(pattern) #Index of pattern
    
    occList = [] #List of positions of occurrences in which pattern is found in text.
    
    while j < len(text):
        if pattern[k] == text[j]:
            j = j+1
            k = k+1
            if k == len(pattern):
                occList.append(j-k)
                k = index[k]
        else:
            k = index[k]
            if k < 0 :
                j = j+1
                k = k+1
    return occList

In [4]:
text = "from the plane to the fuckin' helicopter yeah"

pattern = "he"
print('Text : "{}"\nPattern: "{}"\nOccurences of pattern found at positions : {}'
      .format(text,pattern,kmpSearch(text,pattern)))

Text : "from the plane to the fuckin' helicopter yeah"
Pattern: "he"
Occurences of pattern found at positions : [6, 19, 30]


#### Efficency of search algorithm
Assuming the prior existence of the index, the search portion of the KMP algorithm has **complexity** *O(n)*, where n is the length of S.

Except for the fixed overhead incurred in entering and exiting the function, all the computations are performed in the while loop. The loop can execute at most 2n times, since at each iteration it executes one of the **two branches** in the loop. 

The first branch invariably increases i and does not change m.  The second branch adds i-index\[i\] to m and this is always a positive number (because we saw above that index\[i\] < i). At the same time, the second branch leaves m+i unchanged, for m gets i-index\[i\] added to it, and immediately after index\[i\] gets assigned as the new value of i.

The loop end when m+i = n, so each branch of the loop can be reached at most n times since that the branches increase respectively either m+i or m with m <= m+i. So if m=n then m+i >= n and, since m+i is increased by one unit for each iteration of first branch, m+i = n is true at some point in the past.

The loop is therefore executed a maximum of 2n times and the time complexity is *O(n)*.

#### Overall complexity
Since the two portions of the algorithm have, respectively, complexities of *O(k)* and *O(n)*, the complexity of the overall algorithm is *O(n + k)*, compared to naive algorithm complexity *O(kn)*. 

## References
https://en.wikipedia.org/wiki/Knuth%E2%80%93Morris%E2%80%93Pratt_algorithm